In [86]:
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
raw_data = pd.read_csv('data/bikes.csv')
# drop repeated columns
raw_data = raw_data.drop(columns=['site-2', 'product_id-2'])
raw_data.head()

# TODO: fix table merge code to avoid repeated columns

,bike_type,site,product_id,href,brand,description,price,msrp,rear_pannier_rack,shifter_brake_levers,...,shifters,rear_hub,frame_seatpost,handlebar_height,shifters_brake_lever,groupset_manufacturer,cog,saddle,frame_fork,derailleur
0,road,competitive,PIN004H,/pinarello-dogma-f10-sram-red-etap-complete-ro...,Pinarello,Dogma F10 SRAM Red eTap Complete Road Bike - 2018,12000.0,12000.0,NaN,NaN,...,SRAM Red eTap,NaN,NaN,NaN,NaN,NaN,NaN,Fizik Aliante R7,NaN,NaN
1,road,competitive,PIN005H,/pinarello-dogma-f10-disk-dura-ace-9120-comple...,Pinarello,Dogma F10 Disk Dura-Ace 9120 Complete Road Bike,10000.0,10000.0,NaN,NaN,...,Shimano Dura-Ace R9120,NaN,NaN,NaN,NaN,NaN,NaN,Fizik Aliante R7,NaN,NaN
2,road,competitive,FCB000A,/factor-bike-o2-disc-sram-red-etap-hrd-complet...,Factor Bike,O2 Disc SRAM Red eTap HRD Complete Road Bike -...,9999.0,9999.0,NaN,NaN,...,SRAM Red eTap,NaN,NaN,NaN,NaN,NaN,NaN,Fizik Arione R1,NaN,NaN
3,road,competitive,FCB000E,/factor-bike-one-sram-etap-complete-road-bike?...,Factor Bike,One SRAM Etap Complete Road Bike,8750.0,8750.0,NaN,NaN,...,SRAM Red eTap,NaN,NaN,NaN,NaN,NaN,NaN,Fizik Arione R1,NaN,NaN
4,road,competitive,FCB0008,/factor-bike-one-david-millar-special-edition-...,Factor Bike,One David Millar Special Edition eTap Road Bike,8999.0,13499.0,NaN,NaN,...,SRAM Red eTap,NaN,NaN,NaN,NaN,NaN,NaN,Brooks Cambium C13 Chpt./// Limited Edition,NaN,NaN


### Preliminary Cleaning

In [3]:
# Use description to populate bike_types
def parse_desc(desc):
    BIKE_TYPES_LIST = [  # order matters for fork, frame, kid, girl, and bmx as qualifiers
        'frame', 'fork', 'kid', 'girl', 'e-bike', 'electric', 'folding', 'balance',
        'push', 'trailer', 'boy', 'bmx', 'city', 'commuter', 'comfort', 'fitness',
        'cruiser', 'fat', 'triathlon', 'road', 'touring', 'urban',
        'track', 'adventure', 'mountain', 'cyclocross', 'hybrid',
        'gravel', 'pavement', 'gravel', 'cargo', 'hardtail', 'singlespeed'
    ]
    
    # relabel some string literals
    desc = desc.lower()
    desc = desc.replace('moutain', 'mountain')  # fix typo
    desc = desc.replace('racing', 'road')  # map to road
    desc = desc.replace('suspension', 'mountain')  # map to mountain
    desc = desc.replace('commute', 'commuter')
    desc = desc.replace('step-through', 'urban')

    for bike_type in BIKE_TYPES_LIST:
        if re.search(re.escape(bike_type), desc, re.IGNORECASE):
            return bike_type

    return np.NaN

print(len(raw_data[raw_data.bike_type.isnull()]))

for idx in raw_data[raw_data.bike_type.isnull()].index:
    raw_data.loc[idx, 'bike_type'] = parse_desc(raw_data.description[idx])
    
raw_data[raw_data.bike_type.isnull()]

614


,bike_type,site,product_id,href,brand,description,price,msrp,rear_pannier_rack,shifter_brake_levers,...,shifters,rear_hub,frame_seatpost,handlebar_height,shifters_brake_lever,groupset_manufacturer,cog,saddle,frame_fork,derailleur
230,NaN,performance,7000000000000043340,/shop/bikes-frames/bike-se-big-ripper-29-19-31...,SE,SE Big Ripper 29 - 2019,699.00,699.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SE Flyer Seat w/ Custom SE Bottle Opener Brace,NaN,NaN
285,NaN,performance,7000000000000043335,/shop/bikes-frames/bike-se-big-flyer-29-19-31-...,SE,SE Bikes Big Flyer 29 - 2019,549.00,549.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SE Big Flyer Seat w/ Super Grippy Top Cover,NaN,NaN
292,NaN,performance,7000000000000033771,/shop/bikes-frames/bike-se-dblks-bg-rip29-19-3...,SE,SE Bikes Dblocks Big Ripper 29 - 2019,699.00,699.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raise It Up Flyer Seat w/ Custom SE Bottle Ope...,NaN,NaN
343,NaN,performance,7000000000000043352,/shop/bikes-frames/bike-se-floval-flyer24-19-3...,SE,SE Bikes Floval Flyer 24 - 2019,699.00,699.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SE Racing Pivotal Race Seat,NaN,NaN
600,NaN,wiggle,100417480,/vitus-sommet-crx-fs-bike-carbon-sram-x1-1x11,Vitus,Vitus Sommet CRX FS Bike - Carbon Sram X1 1x11,3137.99,4482.84,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SRAM,NaN,NaN,NaN,NaN
793,NaN,wiggle,100888891,/commencal-meta-ht-am-essential-2019-bike,Commencal,Commencal Meta HT AM Essential (2019) Bike,1799.00,1799.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"RIDE ALPHA 278 mm, CrMo rails",NaN,NaN
795,NaN,wiggle,100805008,/gt-aggressor-sport-2019-bike,GT,GT Aggressor Sport (2019) Bike,399.00,399.00,NaN,NaN,...,NaN,Formula QR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
800,NaN,wiggle,100805062,/gt-gtr-comp-2019-bike,GT,GT GTR Comp (2019) Bike,999.00,999.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Shimano,NaN,GT road saddle,NaN,NaN
810,NaN,wiggle,100805033,/gt-avalanche-comp-2019-bike,GT,GT Avalanche Comp (2019) Bike,599.00,599.00,NaN,NaN,...,NaN,"Formula QR, 6 bolt",NaN,NaN,NaN,Shimano,NaN,NaN,NaN,NaN
813,NaN,wiggle,100805028,/gt-avalanche-elite-2019-bike,GT,GT Avalanche Elite (2019) Bike,729.00,729.00,NaN,NaN,...,NaN,"Formula QR, center lock",NaN,NaN,NaN,Shimano,NaN,WTB Silverado Sport,NaN,NaN


In [4]:
# Parse description to populate model_year
def parse_model_year(desc):
    result = re.search(pattern=r'20[0-9]{2}', string=desc)
    if result is None:
        return np.NaN
    year = int(result.group(0))
    return year if year < 2021 else np.NaN  # avoid '20.75' in href parsing

print('Adding model year column.')
raw_data['model_year'] = raw_data.description.apply(parse_model_year)

print(f'There are currently {len(raw_data[raw_data.model_year.isnull()])} missing model year data.')

for idx in raw_data[raw_data.model_year.isnull()].index:
    raw_data.loc[idx, 'model_year'] = parse_model_year(raw_data.href[idx])

print(f'There are now {len(raw_data[raw_data.model_year.isnull()])} missing model year data.')

Adding model year column.
There are currently 659 missing model year data.
There are now 627 missing model year data.


In [5]:
raw_data.model_year.value_counts()

2019.0    738
2018.0    638
2017.0    202
2016.0     36
2015.0     16
2020.0      9
2013.0      1
2012.0      1
Name: model_year, dtype: int64

In [6]:
# Preliminary fill some NaNs from redudant columns
raw_data.seatpost.fillna(raw_data.seat_post, inplace=True)
raw_data.handlebar.fillna(raw_data.handlebars, inplace=True)
raw_data.chain.fillna(raw_data.chainset, inplace=True)

# merge wheelset data
raw_data.wheelset.fillna(raw_data.wheels, inplace=True)
raw_data.wheelset.fillna(raw_data.wheel, inplace=True)
raw_data.wheelset.fillna(raw_data.wheels_and_tires, inplace=True)

# merge weight data
raw_data.weight.fillna(raw_data.net_weight, inplace=True)
raw_data.weight.fillna(raw_data.bike_weight, inplace=True)
raw_data.weight.fillna(raw_data.frame_weight, inplace=True)

# merge bike_type data including commuter, cyclocross, mtb, road, touring
raw_data.bike_type.fillna(raw_data.best_use, inplace=True)
raw_data.bike_type.fillna(raw_data.recommended_use, inplace=True)

for idx in raw_data[raw_data.bike_type.isnull()].index:
    if raw_data.loc[idx, 'commuter'] == 'Yes':
        bike_type = 'commuter'
    elif raw_data.loc[idx, 'cyclocross'] == 'Yes':
        bike_type = 'cyclocross'
    elif raw_data.loc[idx, 'mtb'] == 'Yes':
        bike_type = 'mountain'
    elif raw_data.loc[idx, 'road'] == 'Yes':
        bike_type = 'road'
    elif raw_data.loc[idx, 'touring'] == 'Yes':
        bike_type = 'touring'
    else:
        bike_type = np.NaN
        
    raw_data.loc[idx, 'bike_type'] = bike_type

# merge frame data
raw_data.frame_material.fillna(raw_data.frame, inplace=True)
raw_data.frame_material.fillna(raw_data.frame_and_fork, inplace=True)
raw_data.frame_material.fillna(raw_data.material, inplace=True)

# merge fork data
raw_data.fork_material.fillna(raw_data.fork, inplace=True)
raw_data.fork_material.fillna(raw_data.forks, inplace=True)
raw_data.fork_material.fillna(raw_data.frame_and_fork, inplace=True)

# merge derailleur data
raw_data.front_derailleur.fillna(raw_data.derailleur, inplace=True)
raw_data.front_derailleur.fillna(raw_data.derailleurs, inplace=True)
raw_data.rear_derailleur.fillna(raw_data.derailleur, inplace=True)
raw_data.rear_derailleur.fillna(raw_data.derailleurs, inplace=True)
raw_data.front_derailleur.fillna(raw_data.bicycle_drivetrain, inplace=True)
raw_data.rear_derailleur.fillna(raw_data.bicycle_drivetrain, inplace=True)

# merge crankset data
raw_data.crankset.fillna(raw_data.cranks, inplace=True)
raw_data.crankset.fillna(raw_data.chainwheel, inplace=True)
raw_data.crankset.fillna(raw_data.crank_set, inplace=True)
raw_data.crankset.fillna(raw_data.bicycle_drivetrain, inplace=True)
raw_data.crankset.fillna(raw_data.drivetrain, inplace=True)
raw_data.crankset.fillna(raw_data.gearset, inplace=True)

# merge cassette data
raw_data.cassette.fillna(raw_data.cassette_rear_cogs, inplace=True)
raw_data.cassette.fillna(raw_data.rear_cogs, inplace=True)
raw_data.cassette.fillna(raw_data.cog, inplace=True)
raw_data.cassette.fillna(raw_data.cog_freewheel, inplace=True)
raw_data.cassette.fillna(raw_data.cogset, inplace=True)
raw_data.cassette.fillna(raw_data.freewheel_cassette, inplace=True)

# merge shifters data
raw_data.shifters.fillna(raw_data.shifter, inplace=True)
raw_data.shifters.fillna(raw_data.bicycle_drivetrain, inplace=True)
raw_data.shifters.fillna(raw_data.derailleur_shifter, inplace=True)
raw_data.shifters.fillna(raw_data.derailleur_shifters, inplace=True)
raw_data.shifters.fillna(raw_data.shifter_brake_levers, inplace=True)
raw_data.shifters.fillna(raw_data.brake_shift_levers, inplace=True)  # should come after shifter
raw_data.shifters.fillna(raw_data.shifters_brake_lever, inplace=True)
raw_data.shifters.fillna(raw_data.shifters_brake_levers, inplace=True)

# merge brakes data
raw_data.brake_type.fillna(raw_data.brakes, inplace=True)
raw_data.brake_type.fillna(raw_data.brake_calipers, inplace=True)
raw_data.brake_type.fillna(raw_data.calipers, inplace=True)
raw_data.brake_type.fillna(raw_data.front_and_rear_brakes, inplace=True)
raw_data.brake_type.fillna(raw_data.brake_levers, inplace=True)  # should come after brake_calipers
raw_data.brake_type.fillna(raw_data.brake_system, inplace=True)  # should come after brake_levers
raw_data.brake_type.fillna(raw_data.rear_brake, inplace=True)

In [7]:
# Custom cleaner for citybike other bike_types -> ebike
print(f'There are {len(raw_data[(raw_data.bike_type == "other") & (raw_data.site == "citybikes")])} other citybikes')
for idx in raw_data[(raw_data.bike_type == 'other') & (raw_data.site == 'citybikes')].index:
    raw_data.loc[idx, 'bike_type'] = 'ebike'

print(f'There are now {len(raw_data[(raw_data.bike_type == "other") & (raw_data.site == "citybikes")])} other citybikes')

There are 28 other citybikes
There are now 0 other citybikes


### Identify Initial Features

In [8]:
print('Number of products:', len(raw_data))
col_counts = raw_data.count()
col_counts = col_counts[col_counts > 500]
print(f'Starting with {len(col_counts)} columns.')
col_counts.sort_index()

Number of products: 2268
Starting with 37 columns.


bike_type           2237
bottom_bracket      1196
brake_type          1959
brakes              1427
brand               2268
cassette            1655
chain               1647
crankset            1736
description         2268
fork                1880
fork_material       1908
frame               1527
frame_material      2061
front_derailleur     674
grips                850
handlebar           1782
handlebars           954
headset             1042
href                2268
hubs                 689
model_year          1641
msrp                2268
pedals              1111
price               2268
product_id          2268
rear_derailleur     1385
rear_shock           516
rims                 835
saddle              1795
seat_post           1086
seatpost            1730
shifters            1302
site                2268
stem                1740
tires               1797
wheel_size           701
wheelset             810
dtype: int64

In [9]:
# Focus on columns with sufficient data
data = raw_data[col_counts.index]
print(f'Initializing with {data.shape} shape.')

# Start removing features that aren't useful, duplicates, or can't easily clean into useful format

no_use_feat = ['rims', 'pedals', 'wheelset', 'bottom_bracket', 'hubs',
              'wheel_size', 'tires', 'grips', 'rear_shock',
              'stem', 'headset', 'saddle', 'href', 'description',
              'seat_post', 'brakes', 'frame', 'handlebars',
              'site', 'product_id', 'msrp', 'fork']
data = data.drop(columns=no_use_feat)
print(f'Data munging with {data.shape} shape.')

Initializing with (2268, 37) shape.
Data munging with (2268, 15) shape.


### Normalize bike_type values

In [10]:
# clean up bike_type labels and drop specified categories
def bike_type_replace(elem):
    bike_mapper = {
        'mountain_bikes': 'mountain',
        'Mountain Biking': 'mountain',
        'Recreational Cycling': 'urban',
        'Urban Cycling': 'urban',
        'Road Cycling': 'road',
        'Cycling': 'other',
        'road_bikes': 'road',
        'path_pavement_bikes': 'urban',
        'urban_bikes': 'urban',
        'boy': np.NaN,
        'kid': np.NaN,  # prep for dropping
        'youth': np.NaN,  # prep for dropping
        'childrens': np.NaN,  # prep for dropping
        'bmx_bikes': 'bmx',
        'commuter_urban': 'urban',
        'electric': 'ebike',
        'electric_bikes': 'ebike',
        'e-bike': 'ebike',
        'girl': np.NaN,  # prep for dropping
        'gravel': 'cyclocross',
        'fat': 'mountain',
        'frame': np.NaN,  # prep for dropping
        'fork': np.NaN, # prep for dropping
        'Bike Touring': 'touring',
        'Bike Commuting': 'urban',
        'Road Cycling, Bikepacking': 'touring',
        'Fitness': 'commuter',
        'fitness': 'commuter',
        'pavement': 'commuter',
        'hardtail': 'mountain',
        'balance': np.NaN,
        'push': np.NaN,
        'cargo': np.NaN,
        'trailer': np.NaN,
        'Bikepacking, Bike Touring': 'touring',
        'Mountain Biking, Bikepacking': 'touring',
        'Bikepacking, Mountain Biking': 'touring',
        'Cyclocross': 'cyclocross',
        'Bikepacking, Road Cycling': 'touring',
        'Bike Touring, Bikepacking': 'touring',
        'cruiser': 'comfort',
        'city': 'urban',
        'adventure': 'comfort',
        'triathlon': np.NaN,  # dropping due to insufficient data count (< 5)
        'track': np.NaN  # dropping due to insufficient data count (< 5)
    }
    return bike_mapper.get(elem, elem)
    

data.bike_type = data.bike_type.apply(bike_type_replace)
data.bike_type = data.bike_type.dropna()
data.bike_type.value_counts()

mountain       781
road           438
urban          236
bmx            132
ebike          129
cyclocross      71
comfort         46
hybrid          40
touring         30
commuter        23
folding         12
other            3
singlespeed      1
Name: bike_type, dtype: int64

In [11]:
# # Looks like ebike, cyclocross, mountain, road are on average pricy
# one_hot = pd.get_dummies(data['bike_type'])[['ebike', 'cyclocross', 'mountain', 'road']]
# data['ebike'] = one_hot.ebike
# data['cyclocross'] = one_hot.cyclocross
# data['mountain'] = one_hot.mountain
# data['road'] = one_hot.road
# # data['touring'] = one_hot.touring
# data.head()

### Normalize brand values

In [12]:
# Clean up brand names
def brand_replace(brand):
    brand = brand.replace(' Bikes', '')
    brand = brand.replace(' Bike', '')
    brand = brand.replace(' Bicycles', '')
    brand = brand.replace(' Electric', '')
    brand = brand.replace(' S-Works', '')
    brand = brand.replace(' Cycles', '')
    brand = brand.replace(' Turbo', '')
    brand = brand.replace('S-Works', 'Specialized')
    return brand

data.brand = data.brand.apply(brand_replace)

# Resolve truncated brand names
data.loc[data.brand == 'Santa', 'brand'] = 'Santa Cruz'  # Standardize Santa -> Santa Cruz
data.loc[data.brand == 'We', 'brand'] = 'We The People'
data.loc[data.brand == 'De', 'brand'] = 'De Rosa'

In [13]:
brands = data.brand.unique().tolist()
brands.sort()
brands

['Adams',
 'Alchemy',
 'Benno',
 'Bianchi',
 'Bike',
 'Blank',
 'Bobbin',
 'Borealis',
 'Brand-X',
 'Breezer',
 'Brompton',
 'Burley',
 'Cannondale',
 'Cinelli',
 'Cleary',
 'Co-Motion',
 'Co-op',
 'Colnago',
 'Commencal',
 'Cube',
 'DK',
 'Dahon',
 'De Rosa',
 'DeVinci',
 'Diamondback',
 'Early Rider Limited',
 'Eastern',
 'Electra',
 'ElliptiGO',
 'Evil',
 'Factor',
 'Fit',
 'Framed',
 'Fuji',
 'GHOST',
 'GT',
 'Gazelle',
 'Genesis',
 'GoCycle',
 'Haibike',
 'Haro',
 'Hellafaster',
 'IZIP',
 'Ibis',
 'Jamis',
 'Juliana',
 'Kalkhoff',
 'Kestrel',
 'Kiddimoto',
 'Kink',
 'Liv',
 'Marin',
 'Masi',
 'Miir',
 'Mongoose',
 'Nashbar',
 'Niner',
 'Nirve',
 'Nukeproof',
 'Octane',
 'Orro',
 'Oyama',
 'PK',
 'Pinarello',
 'Pivot',
 'Premium',
 'Prevelo',
 'Pure',
 'Raleigh',
 'Ridgeback',
 'Ridley',
 'RockShox',
 'Rondo',
 'Ruption',
 'SE',
 'STROMER',
 'Salsa',
 'Santa Cruz',
 'Saracen',
 'Spank',
 'Specialized',
 'Stolen',
 'Storck',
 'Strider',
 'Tern',
 'Tuesday',
 'Van',
 'Vitus',
 'We Th

### Normalize Frame Material Values

In [14]:
def material_replace(elem):
    MATERIALS_LIST = [
        'carbon', 'aluminum', 'aluminium', 'steel', 'alloy',
        'titanium', 'chromoly', 'crmo', 'cr-mo', 'hi-ten',
        'alluminum'
    ]
    MATERIALS_DICT = {
        'carbon': 'carbon',
        'aluminium': 'aluminum',
        'aluminum': 'aluminum',
        'alloy': 'alloy',
        'steel': 'steel',
        'titanium': 'titanium',
        'chromoly': 'chromoly',
        'crmo': 'chromoly',
        'alluminum': 'aluminum',
        'cr-mo': 'chromoly',
        'hi-ten': 'steel'
    }
    try:
        for material in MATERIALS_LIST:
            if re.search(re.escape(material), elem, re.IGNORECASE):
                return MATERIALS_DICT[material]
    except TypeError:
        return np.NaN
    return np.NaN

data.frame_material = data.frame_material.apply(material_replace)
data.frame_material.value_counts()

carbon      750
aluminum    673
alloy       332
steel       142
chromoly     91
titanium      4
Name: frame_material, dtype: int64

In [15]:
# one_hot = pd.get_dummies(data['frame_material'])
# data['carbon_frame'] = one_hot.carbon
# data['titanium_frame'] = one_hot.titanium
# data

### Normalize Handlebar Values

In [16]:
data['handlebar_material'] = data.handlebar.apply(material_replace)
data.handlebar_material.value_counts()

alloy       749
carbon      161
aluminum    120
steel       109
chromoly     37
Name: handlebar_material, dtype: int64

In [17]:
# one_hot = pd.get_dummies(data['handlebar'])
# data['carbon_handlebar'] = one_hot.carbon
# data

### Groupset Category Ranks
Plan was to use average price to determine rank but found site that has something similar: https://www.tredz.co.uk/component-buyers-guides/road-groupsets

https://www.wheelies.co.uk/buying-guide/bike-groupset-guide

https://www.bikeexchange.com.au/blog/road-bike-groupsets-what-to-know

In [18]:
GROUPSET_RANKING = {
    'shimano claris': 1,
    'shimano sora': 2,
    'shimano tiagra': 2,
    'shimano 105': 3,
    'shimano ultegra': 4,
    'shimano ultegra di2': 5,
    'shimano dura-ace': 5,
    'shimano dura-ace di2': 6,
    'sram apex': 2,
    'sram rival': 3,
    'sram s700': 3,
    'sram force': 4,
    'sram red': 5,
    'sram red etap': 6,
    'campagnolo veloce': 1,
    'campagnolo centaur': 3,
    'campagnolo athena': 2.5,
    'campagnolo potenza': 4,
    'campagnolo chorus': 4,
    'campagnolo athena eps': 3.5,
    'campagnolo record': 5,
    'campagnolo chorus eps': 5,
    'campagnolo super record': 5.25,
    'campagnolo record eps': 5.5,
    'campagnolo super record eps': 6,
    'shimano tourney': 0.5,
    'shimano altus': 1,
    'shimano acera': 1.5,
    'shimano alivio': 2,
    'shimano deore': 2.75,
    'shimano slx': 3,
    'shimano zee': 3,
    'shimano deore xt': 4,
    'shimano xt': 4, #?
    'shimano saint': 4.5,
    'shimano xt di2': 4,
    'shimano xtr': 5,
    'shimano xtr di2': 6,
    'sram x3': 0.5,
    'sram x4': 1.5,
    'sram x5': 2,
    'sram x7': 2.25,
    'sram x9': 2.6,
    'sram nx': 3,
    'sram gx dh': 3,
    'sram gx': 3.25,
    'sram gx eagle': 3.5,
    'sram x1': 3.75,
    'sram xO1 dh': 4.25,
    'sram xO': 4.0,
    'sram xO1': 4.25,
    'sram xx': 4.5,
    'sram xx1': 5,
    'sram xO1 eagle': 4.75,
    'sram xx1 eagle': 6,
    'sram via': 3.5,
    'shimano 7-speed': 0.5,
    'shimano 8-speed': 1,
    'shimano 9-speed': 2,
    'shimano 10-speed': 2.5,
    'sram 8-speed': 0.5,
    'sram 9-speed': 1,
    'sram 10-speed': 2,
}

GROUPSETS_MAP = {
    'claris': 'shimano claris',
    'sora': 'shimano sora',
    'tiagra': 'shimano tiagra',
    '105': 'shimano 105',
    'ultegra': 'shimano ultegra',
    'ultegra di2': 'shimano ultegra di2',
    'dura-ace': 'shimano dura-ace',
    'dura-ace di2': 'shimano dura-ace di2',
    'apex': 'sram apex',
    'rival': 'sram rival',
    's700': 'sram s700',
    'force': 'sram force',
    'red': 'sram red',
    'red etap': 'sram red etap',
    'veloce': 'campagnolo veloce',
    'centaur': 'campagnolo centaur',
    'athena': 'campagnolo athena',
    'potenza': 'campagnolo potenza',
    'chorus': 'campagnolo chorus',
    'athena eps': 'campagnolo athena eps',
    'record': 'campagnolo record',
    'chorus eps': 'campagnolo chorus eps',
    'super record': 'campagnolo super record',
    'record eps': 'campagnolo record eps',
    'super record eps': 'campagnolo super record eps',
    'tourney': 'shimano tourney',
    'altus': 'shimano altus',
    'acera': 'shimano acera',
    'alivio': 'shimano alivio',
    'deore': 'shimano deore',
    'slx': 'shimano slx',
    'zee': 'shimano zee',
    'deore xt': 'shimano deore xt',
    'xt': 'shimano deore xt',
    'saint': 'shimano saint',
    'xt di2': 'shimano xt di2',
    'xtr': 'shimano xtr',
    'xtr di2': 'shimano xtr di2',
    'x3': 'sram x3',
    'x4': 'sram x4',
    'x5': 'sram x5',
    'x7': 'sram x7',
    'x9': 'sram x9',
    'nx': 'sram nx',
    'gx dh': 'sram gx dh',
    'gx': 'sram gx',
    'gx eagle': 'sram gx eagle',
    'x1': 'sram x1',
    'xO1 dh': 'sram xO1 dh',
    'xO': 'sram xO',
    'xO1': 'sram xO1',
    'xx': 'sram xx',
    'xx1': 'sram xx1',
    'xO1 eagle': 'sram xO1 eagle',
    'xx1 eagle': 'sram xx1 eagle',
    'microshift': 'microshift',
    'shimano 2400': 'shimano claris',
    'shimano acero': 'shimano acera',
    'shimano r7000': 'shimano 105',
    'shimano 9150': 'shimano dura-ace di2',
    'shimano 9100': 'shimano dura-ace',
    'shimano 9070': 'shimano dura-ace di2',
    'shimano 9000': 'shimano dura-ace',
    'shimano r8000': 'shimano ultegra',
    'shimano r8050': 'shimano ultegra di2',
    'shimano 6800': 'shimano ultegra',
    'shimano 6870': 'shimano ultegra di2',
    'shimano 5800': 'shimano 105',
    'shimano 4700': 'shimano tiagra', 
    'shimano r3000': 'shimano sora',
    'shimano tx800': 'shimano tourney',
    'shimano t4000': 'shimano alivio',
    'shimano ty510': 'shimano tourney',
    'shimano ty700': 'shimano tourney',
    'shimano fd-ty500': 'shimano tourney',
    'shimano fd-m190': 'shimano tourney',
    'shimano mt400': 'shimano alivio',
    'shimano fd-ty710': 'shimano tourney',
    'shimano tz31': 'shimano tourney',
    'shimano tz-30': 'shimano tourney',
    'shimano tz30': 'shimano tourney',
    'shimano a070': 'shimano tourney',
    'shimano st-ef51': 'shimano tiagra',
    'shimano fd-m191': 'shimano altus',
    'sram etap': 'sram red etap',
    'sram x01 eagle': 'sram xO1 eagle',
    'sram x01': 'sram xO1',
    'sram eagle x01': 'sram xO1 eagle',
    'sram eagle xO1': 'sram xO1 eagle',
    'sram eagle xo1': 'sram xO1 eagle',
    'sram via': 'sram via',
    'shimano tx 8000': 'shimano tourney',
    'shimano tx-8000': 'shimano tourney',
    'shimano tx8000': 'shimano deore',
    'shimano rd-m7000': 'shimano slx',
    'shimano m7000': 'shimano slx',
    'shimano rd-m8000': 'shimano deore xt',
    'shimano m8000': 'shimano deore xt',
    'shimano tz-50': 'shimano tourney',
    'shimano tz50': 'shimano tourney',
    'shimano ty300': 'shimano tourney',
    'shimano ty-300': 'shimano tourney',
    'shimano tx-35': 'shimano tourney',
    'shimano tx35': 'shimano tourney',
    'shimano rd-m2000': 'shimano altus',
    'shimano rdm2000': 'shimano altus',
    'shimano rd m2000': 'shimano altus',
    'shimano ft30': 'shimano tourney',
    'shimano ft 30': 'shimano tourney',
    'shimano ft-30': 'shimano tourney',
    'shimano rd-5800': 'shimano 105',
    'shimano rd 5800': 'shimano 105'
}

# Get groupset keys as list for regex search
GROUPSETS_LIST = list(GROUPSETS_MAP.keys())
GROUPSETS_LIST.sort()
GROUPSETS_LIST.reverse()
GROUPSETS_LIST[:11]  # show first 10 to confirm compound names come first

['zee',
 'xx1 eagle',
 'xx1',
 'xx',
 'xtr di2',
 'xtr',
 'xt di2',
 'xt',
 'xO1 eagle',
 'xO1 dh',
 'xO1']

### Normalize Front Derailleur Values

In [32]:
def derailleur_replace(desc, return_desc=True):
    try:
        for groupset in GROUPSETS_LIST:
            # remove ','
            desc = desc.replace(',', '')
            # Regex matching
            if re.search(re.escape(groupset), desc, re.IGNORECASE):
                return GROUPSETS_MAP[groupset]
            # Extra custom cleaning
            others = {
                'X01 Eagle': 'sram xO1 eagle',
                'SEAM X01 Eagle': 'sram xO1 eagle',
                'Sram A1': 'sram apex'
            }
            if desc in others.keys():
                return others[desc]
            
    except TypeError:
        pass        
    except AttributeError:
        pass
    return desc if return_desc else np.NaN

data['fd_groupset'] = data.front_derailleur.apply(derailleur_replace, return_desc=False)

In [33]:
data.fd_groupset.value_counts()

shimano 105                97
shimano ultegra            89
shimano tourney            84
shimano altus              78
shimano tiagra             49
microshift                 38
shimano deore              28
shimano sora               27
shimano claris             25
shimano ultegra di2        20
sram red etap              18
shimano acera              16
shimano dura-ace           15
shimano alivio             12
shimano slx                10
shimano dura-ace di2        9
sram x1                     7
sram x5                     4
shimano deore xt            4
sram force                  3
sram x3                     2
campagnolo record           2
sram x7                     2
sram via                    1
sram red                    1
sram rival                  1
sram apex                   1
campagnolo super record     1
Name: fd_groupset, dtype: int64

### Normalize Rear Derailleur Values

In [36]:
data['rd_groupset'] = data.rear_derailleur.apply(derailleur_replace, return_desc=False)

In [37]:
data.rd_groupset.value_counts()

shimano tourney            122
sram gx eagle              105
sram nx                    104
sram xO1 eagle             103
shimano deore              100
shimano 105                 99
shimano ultegra             96
shimano deore xt            78
shimano acera               68
shimano altus               64
shimano tiagra              57
shimano alivio              43
sram gx                     41
shimano slx                 37
shimano sora                28
shimano claris              24
shimano xtr                 22
sram apex                   19
shimano ultegra di2         19
sram xx1 eagle              18
sram x1                     18
sram red etap               18
shimano dura-ace            14
sram force                  14
sram xO1                    11
shimano dura-ace di2         9
sram rival                   9
sram x5                      5
sram x7                      4
sram x4                      4
sram xx1                     2
campagnolo record            2
sram x3 

### Normalize cassette values

In [54]:
# First pass using derailleur groupset logic
data['cassette_groupset'] = data.cassette.apply(derailleur_replace, return_desc=False)

def cassette_replace(desc, return_desc=True):    
    CASSETTE_MAP = {
        'sunrace': 'sunrace',
        'shimano hg500': 'shimano tiagra',
        'shimano hg 500': 'shimano tiagra',
        'shimano hg-500': 'shimano tiagra',
        'sram pg-1130': 'sram rival',
        'sram pg1130': 'sram rival',
        'sram pg 1130': 'sram rival',
        'sram xg 1150': 'sram gx',
        'sram xg1150': 'sram gx',
        'sram xg-1150': 'sram gx',
        'sram xg-1175': 'sram gx',
        'sram xg 1175': 'sram gx',
        'sram xg-175': 'sram gx',
        '1295 eagle': 'sram xO1 eagle',
        '1275 eagle': 'sram gx eagle',
        'sram xg-1190': 'sram red',
        'sram xg1190': 'sram red',
        'sram xg 1190': 'sram red',
        'shimano hg50': 'shimano sora',
        'shimano hg 50': 'shimano sora',
        'shimano hg-50': 'shimano sora',
        'shimano hg200': 'shimano tourney',
        'shimano hg 200': 'shimano tourney',
        'shimano hg-200': 'shimano tourney',
        'shimano hg-20': 'shimano tourney',
        'shimano hg 20': 'shimano tourney',
        'shimano hg20': 'shimano tourney',
        'shimano hg41': 'shimano acera',
        'shimano hg 41': 'shimano acera',
        'shimano hg-41': 'shimano acera',
        'flip flop': 'single speed',
        '22t steel': 'single speed',
        'fixed': 'single speed',
        'freewheel': 'single speed',
        'shimano hg31': 'shimano altus',
        'shimano hg-31': 'shimano altus',
        'shimano hg 31': 'shimano altus',
        'shimano hg-700': 'shimano 105',
        'shimano hg 700': 'shimano 105',
        'shimano hg700': 'shimano 105',
        'sram pg-1170': 'sram force',
        'sram pg1170': 'sram force',
        'sram pg 1170': 'sram force',
        'shimano hg400': 'shimano 9-speed',
        'shimano hg-400': 'shimano 9-speed',
        'shimano hg 400': 'shimano 9-speed',
        'hg 400': 'shimano 9-speed',
        'hg-400': 'shimano 9-speed',
        'hg400': 'shimano 9-speed',
        'shimano hg62': 'shimano deore',
        'shimano hg-62': 'shimano deore',
        'shimano hg 62': 'shimano deore',
        'shimano hg300': 'shimano alivio',
        'shimano hg-300': 'shimano alivio',
        'shimano hg 300': 'shimano alivio',
        'shimano 9s': 'shimano 9-speed',
        'sram pg970': 'sram 9-speed',
        'x01 eagle': 'sram xO1 eagle',
        'cs5700': 'shimano 105'
    }
    
    SPEEDS_MAP = {
        'shimano.*7.{0,1}sp.{0,3}': 'shimano 7-speed',
        'shimano.*8.{0,1}sp.{0,3}': 'shimano 8-speed',
        'shimano.*9.{0,1}sp.{0,3}': 'shimano 9-speed',
        'shimano.*10.{0,1}sp.{0,3}': 'shimano 10-speed',
        'shimano.*11.{0,1}sp.{0,3}': 'shimano 11-speed',
        'sram.*7.{0,1}sp.{0,3}': 'sram 7-speed',
        'sram.*8.{0,1}sp.{0,3}': 'sram 8-speed',
        'sram.*9.{0,1}sp.{0,3}': 'sram 9-speed',
        'sram.*10.{0,1}sp.{0,3}': 'sram 10-speed',
        'sram.*11.{0,1}sp.{0,3}': 'sram 11-speed',
        'shimano.*5800': 'shimano 105',
        'shimano dura.{0,1}ace': 'shimano dura-ace',
        'sram.*1275': 'sram gx eagle',
        'sram.*pg.{0,1}1230': 'sram gx eagle',
        'sram.*xg.{0,1}1295': 'sram xO1 eagle',
        'sram .*1130': 'sram rival',
        'sram.*1299[ eagle]{0,1}': 'sram xx1 eagle',
        '\d{1,2}t cassett{0,1}e|cog': 'single speed',
        'hg.{0,1}200': 'shimano tourney'
    }
    
    try:
        # prelim clean
        desc = desc.lower()
        desc = desc.replace('cs-', '')
        desc = desc.replace('seam', 'sram')  # fix typo
        
        for cassette in CASSETTE_MAP.keys():
            # Regex literal search
            if re.search(re.escape(cassette), desc, re.IGNORECASE):
                return CASSETTE_MAP[cassette]
        
        # regex alternative search
        for cassette in SPEEDS_MAP.keys():
            if re.search(cassette, desc, re.IGNORECASE):
                return SPEEDS_MAP[cassette]

    except TypeError:
        pass        
    except AttributeError:
        pass
    return desc if return_desc else np.NaN

# Second pass, fillnas when possible using cassette specific logic
for idx in data[data.cassette_groupset.isnull()].index:
    data.loc[idx, 'cassette_groupset'] = cassette_replace(data.cassette[idx], return_desc=False)

In [55]:
data.cassette_groupset.value_counts()

sunrace                    209
sram gx eagle              167
shimano 105                155
shimano tourney            106
sram xO1 eagle              96
single speed                88
shimano tiagra              88
sram rival                  74
shimano ultegra             72
shimano slx                 59
sram nx                     47
shimano 9-speed             37
sram gx                     31
shimano altus               30
shimano deore xt            29
shimano acera               26
shimano 8-speed             22
shimano sora                22
shimano alivio              19
shimano 10-speed            19
shimano dura-ace            16
shimano deore               13
sram red                    13
sram force                  13
sram x1                     12
shimano 7-speed             11
shimano 11-speed             7
sram 9-speed                 7
sram 8-speed                 7
sram xx1 eagle               5
sram 10-speed                5
sram 11-speed                3
sram xO1

### Normalize Crankset Values

In [56]:
data['crankset_material'] = data.crankset.apply(material_replace)

In [57]:
data.crankset_material.value_counts()

alloy       332
steel       120
carbon       89
aluminum     63
chromoly     53
Name: crankset_material, dtype: int64

In [60]:
# First pass using derailleur groupset logic
data['crankset_groupset'] = data.crankset.apply(derailleur_replace, return_desc=False)

def crankset_replace(desc, return_desc=True):
    BRANDS = ['praxis', 'oval', 'race face', 'fsa', 'sram stylo']
    
    try:
        # resolve some typos
        desc = desc.lower()
        desc = desc.replace('raceface', 'race face')
        for brand in BRANDS:
            if re.search(brand, desc, re.IGNORECASE):
                return brand

    except TypeError:
        pass        
    except AttributeError:
        pass
    return desc if return_desc else np.NaN

# Second pass, fillnas when possible using crankset specific logic
for idx in data[data.crankset_groupset.isnull()].index:
    data.loc[idx, 'crankset_groupset'] = crankset_replace(data.crankset[idx], return_desc=False)

In [61]:
data.crankset_groupset.value_counts()

praxis                     111
race face                  104
fsa                         99
sram x1                     98
sram nx                     73
shimano ultegra             55
sram stylo                  45
shimano tourney             39
sram gx eagle               38
sram xO1 eagle              32
shimano deore xt            27
sram x9                     26
sram x3                     25
shimano acera               19
shimano deore               17
sram xx1 eagle              17
sram red                    16
shimano slx                 14
sram gx                     14
shimano tiagra              14
sram xO                     14
shimano 105                 13
shimano dura-ace            10
oval                         9
shimano altus                8
sram rival                   7
sram x7                      7
sram xO1                     7
sram apex                    7
sram force                   6
shimano claris               6
shimano alivio               5
shimano 

### Normalize Brake Type Values

In [63]:
# Categorize brake types per str
def brake_replace(brake):
    MATERIALS_LIST = [
        'hydraulic', 'mechanical', 'rim', 'caliper', 'coaster',
        'disc', 'v-brake', 'u-brake', 'linear pull', 'linear-pull'
    ]
    MATERIALS_DICT = {
        'linear pull': 'linear_pull',
        'linear-pull': 'linear_pull',
        'v-brake': 'vbrake',
        'u-brake': 'ubrake'
    }
    try:
        for material in MATERIALS_LIST:
            if re.search(re.escape(material), brake, re.IGNORECASE):
                return MATERIALS_DICT.get(material, material)
    except TypeError:
        return 'other'
    return 'other'

data.brake_type = data.brake_type.apply(brake_replace)
data.brake_type.value_counts()

hydraulic      971
other          569
mechanical     188
disc           150
vbrake         110
rim             90
caliper         64
coaster         52
linear_pull     42
ubrake          32
Name: brake_type, dtype: int64

### Normalize Seatpost Values

In [64]:
data['seatpost_material'] = data.seatpost.apply(material_replace)

In [65]:
data.seatpost_material.value_counts()

alloy       729
carbon      201
aluminum     66
steel        50
chromoly      3
Name: seatpost_material, dtype: int64

### Normalize Fork Material Values

In [66]:
data.fork_material = data.fork_material.apply(material_replace)

In [67]:
data.fork_material.value_counts()

carbon      523
steel       277
alloy       118
chromoly    106
aluminum     91
Name: fork_material, dtype: int64

### Normalize Chain Values

In [70]:
data['chain_groupset'] = data.chain.apply(derailleur_replace, return_desc=False)

In [71]:
data.chain_groupset.value_counts()

sram x1                    393
sram gx eagle              124
sram x9                     88
sram nx                     79
sram xO1 eagle              67
shimano ultegra             42
shimano dura-ace            18
sram xx1 eagle              17
sram red                    16
shimano 105                 12
shimano slx                 10
shimano deore xt            10
sram x3                      7
shimano xtr                  6
sram gx                      5
sram xO1                     3
shimano tiagra               2
campagnolo record            2
sram force                   1
sram rival                   1
shimano sora                 1
shimano claris               1
campagnolo super record      1
campagnolo chorus            1
shimano tourney              1
sram xx1                     1
Name: chain_groupset, dtype: int64

### Normalize Shifters Values

In [72]:
data['shifter_groupset'] = data.shifters.apply(derailleur_replace, return_desc=False)

In [73]:
data.shifter_groupset.value_counts()

sram gx eagle              112
sram nx                    100
sram xO1 eagle              78
shimano ultegra             73
sram x1                     69
shimano slx                 67
shimano deore               61
shimano altus               59
shimano 105                 57
shimano tourney             42
microshift                  37
shimano tiagra              35
sram x9                     34
shimano deore xt            29
sram gx                     26
shimano acera               21
shimano claris              20
shimano sora                20
sram x7                     18
shimano alivio              17
sram red etap               16
sram xx1 eagle              15
sram apex                   15
shimano ultegra di2         14
sram force                  14
shimano dura-ace            12
sram rival                  11
sram x5                      9
sram s700                    8
sram xO1                     8
shimano dura-ace di2         5
sram x4                      4
sram x3 

In [76]:
munged_df = data.drop(columns=['front_derailleur', 'rear_derailleur', 'handlebar',
                              'cassette', 'crankset', 'seatpost', 'chain', 'shifters'])
munged_df.head()

,bike_type,brand,price,frame_material,model_year,brake_type,fork_material,handlebar_material,fd_groupset,rd_groupset,cassette_groupset,crankset_material,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset
0,road,Pinarello,12000.0,carbon,2018.0,rim,carbon,carbon,sram red etap,sram red etap,sram red,NaN,sram red,NaN,sram red,sram red etap
1,road,Pinarello,10000.0,carbon,NaN,hydraulic,carbon,carbon,shimano dura-ace,shimano dura-ace,shimano dura-ace,NaN,shimano dura-ace,NaN,shimano dura-ace,shimano dura-ace
2,road,Factor,9999.0,carbon,2018.0,hydraulic,carbon,NaN,sram red etap,sram red etap,sram force,NaN,sram red,NaN,sram red,sram red etap
3,road,Factor,8750.0,carbon,NaN,rim,carbon,NaN,sram red etap,sram red etap,sram red,NaN,sram red,NaN,sram red,sram red etap
4,road,Factor,8999.0,carbon,2018.0,rim,carbon,carbon,sram red etap,sram red etap,sram red,NaN,sram red,NaN,sram red,sram red etap


In [77]:
munged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268 entries, 0 to 2267
Data columns (total 16 columns):
bike_type             1942 non-null object
brand                 2268 non-null object
price                 2268 non-null float64
frame_material        1992 non-null object
model_year            1641 non-null float64
brake_type            2268 non-null object
fork_material         1115 non-null object
handlebar_material    1176 non-null object
fd_groupset           644 non-null object
rd_groupset           1364 non-null object
cassette_groupset     1520 non-null object
crankset_material     657 non-null object
crankset_groupset     996 non-null object
seatpost_material     1049 non-null object
chain_groupset        909 non-null object
shifter_groupset      1116 non-null object
dtypes: float64(2), object(14)
memory usage: 283.6+ KB


In [93]:
munged_df.to_csv(f'data/munged/{datetime.date.today()}.csv', sep=',', index=False)

In [94]:
pd.read_csv(f'data/munged/{datetime.date.today()}.csv').head()

,bike_type,brand,price,frame_material,model_year,brake_type,fork_material,handlebar_material,fd_groupset,rd_groupset,cassette_groupset,crankset_material,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset
0,road,Pinarello,12000.0,carbon,2018.0,rim,carbon,carbon,sram red etap,sram red etap,sram red,NaN,sram red,NaN,sram red,sram red etap
1,road,Pinarello,10000.0,carbon,NaN,hydraulic,carbon,carbon,shimano dura-ace,shimano dura-ace,shimano dura-ace,NaN,shimano dura-ace,NaN,shimano dura-ace,shimano dura-ace
2,road,Factor,9999.0,carbon,2018.0,hydraulic,carbon,NaN,sram red etap,sram red etap,sram force,NaN,sram red,NaN,sram red,sram red etap
3,road,Factor,8750.0,carbon,NaN,rim,carbon,NaN,sram red etap,sram red etap,sram red,NaN,sram red,NaN,sram red,sram red etap
4,road,Factor,8999.0,carbon,2018.0,rim,carbon,carbon,sram red etap,sram red etap,sram red,NaN,sram red,NaN,sram red,sram red etap
